In [1]:
import pandas as pd

In [2]:
weather_data = pd.read_csv('Resources/PostgreSQL_weatherdata.csv')  
weather_data.head()

,country,location_name,latitude,longitude,last_updated,temperature_fahrenheit,wind_mph,humidity,cloud,uv_index
0,Afghanistan,Kabul,34.52,69.18,2024-05-16,79.8,8.3,24,30,7.0
1,Albania,Tirana,41.33,19.82,2024-05-16,66.2,6.9,94,75,5.0
2,Algeria,Algiers,36.76,3.05,2024-05-16,73.4,9.4,29,0,5.0
3,Andorra,Andorra La Vella,42.50,1.52,2024-05-16,43.3,7.4,61,100,2.0
4,Angola,Luanda,-8.84,13.23,2024-05-16,78.8,8.1,89,50,8.0


In [3]:
weather_data = weather_data.drop(columns=['wind_mph', 'cloud', 'uv_index'])

In [4]:
weather_data.shape

(36669, 7)

In [5]:
import pandas as pd

def convert_last_updated(date):
    try:
        return pd.to_datetime(date).strftime('%B %Y')
    except:
        return date  

weather_data['last_updated'] = weather_data['last_updated'].apply(convert_last_updated)

average_weather = weather_data.groupby(['country', 'latitude', 'longitude', 'location_name', 'last_updated']).agg(
    average_temperature=('temperature_fahrenheit', 'mean'),
    average_humidity=('humidity', 'mean')
).reset_index()

average_weather['average_temperature'] = average_weather['average_temperature'].round(0).astype(int)
average_weather['average_humidity'] = average_weather['average_humidity'].round(0).astype(int)




In [6]:
average_weather.shape

(1613, 7)

In [7]:
average_weather.head()

,country,latitude,longitude,location_name,last_updated,average_temperature,average_humidity
0,Afghanistan,34.5167,69.1833,Kabul,November 2024,63,24
1,Afghanistan,34.5167,69.1833,Kabul,October 2024,71,22
2,Afghanistan,34.5200,69.1800,Kabul,August 2024,88,21
3,Afghanistan,34.5200,69.1800,Kabul,July 2024,88,18
4,Afghanistan,34.5200,69.1800,Kabul,June 2024,78,25


In [8]:
import ipywidgets as widgets
from IPython.display import display
import folium
from folium.plugins import Fullscreen, MiniMap, Search
import pandas as pd
import geopandas as gpd
from folium import GeoJson

average_weather['last_updated'] = pd.to_datetime(average_weather['last_updated'])
average_weather['month'] = average_weather['last_updated'].dt.strftime('%B %Y')

gdf = gpd.GeoDataFrame(
    average_weather, 
    geometry=gpd.points_from_xy(average_weather['longitude'], average_weather['latitude'])
)

gdf['last_updated'] = gdf['last_updated'].dt.strftime('%Y-%m-%d %H:%M:%S')

gdf['search_field'] = gdf['country'] + " - " + gdf['location_name']

month_dropdown = widgets.Dropdown(
    options=average_weather['month'].unique(),
    value=average_weather['month'].unique()[0],
    description='Date:',
)

recenter_button = widgets.Button(description="Recenter Map")

def get_temperature_color(temp):
    if temp >= 90:
        return 'red'
    elif temp >= 70:
        return 'orange'
    elif temp >= 50:
        return 'lightblue'
    else:
        return 'darkblue'

def create_marker(row):
    color = get_temperature_color(row['average_temperature'])
    marker_icon = folium.Icon(
        color=color, 
        icon='cloud', 
        icon_color='white',
        
    )
    
    return folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"<b>Country:</b> {row['country']}<br>"
              f"<b>Location:</b> {row['location_name']}<br>"
              f"<b>Temperature:</b> {row['average_temperature']}°F<br>"
              f"<b>Humidity:</b> {row['average_humidity']}%",
        icon=marker_icon,  # Use the custom marker icon with the resized size
        tooltip=f"{row['country']} 🌎<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}"
    )


def update_map(change):
    selected_month = month_dropdown.value
    filtered_gdf = gdf[gdf['month'] == selected_month]  

    weather_map = folium.Map(
        location=[average_weather['latitude'].mean(), average_weather['longitude'].mean()],
        zoom_start=2,
        tiles='OpenStreetMap',
        scrollWheelZoom=True
    )
    
    folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}",
    name="Google Terrain",
    attr="Google Maps",
    ).add_to(weather_map)


    Fullscreen().add_to(weather_map)
    MiniMap(toggle_display=True).add_to(weather_map)

    geojson_layer = GeoJson(
        data=filtered_gdf.to_json(),
        name="Weather Data",
        popup=folium.GeoJsonPopup(
            fields=['location_name', 'average_temperature', 'average_humidity'],
            aliases=['Location', 'Avg Temperature °F', 'Avg Humidity %'],
            localize=True
        )
    )
    geojson_layer.add_to(weather_map)

    search = Search(
        layer=geojson_layer,
        search_label="search_field",
        placeholder="Search for Country or Location...",
        position="topright",
        collapsed=False,
            search_bar_style='background-color: rgba(100, 149, 237, 0.4); border: 1px solid #4682B4; border-radius: 8px; color: white;'

    )
    search.add_to(weather_map)

    for _, row in filtered_gdf.iterrows():
        create_marker(row).add_to(weather_map)

    legend_html = """
    <div id="legend" style="
        position: fixed; 
        bottom: 30px; left: 30px; width: 110px; 
        background: rgba(255, 255, 255, 0.6); 
        border-radius: 8px; padding: 10px; 
        box-shadow: 2px 2px 6px rgba(0, 0, 0, 0.4); 
        font-family: Arial, sans-serif; font-size: 12px; 
        z-index:1000;">
        <div id="legend-header" style="cursor: pointer; font-weight: bold;">
            Temperature ▼ 
        </div>
        <div id="legend-content" style="margin-top: 15px;">
            <div><i class="fa fa-sun" style="color: red; margin-right: 7px;"></i> Above 90°F</div>
            <div><i class="fa fa-cloud-sun" style="color: orange; margin-right: 7px;"></i> Above 70°F</div>
            <div><i class="fa fa-cloud" style="color: lightblue; margin-right: 7px;"></i> Above 50°F</div>
            <div><i class="fa fa-snowflake" style="color: darkblue; margin-right: 7px;"></i> Below 50°F</div>
        </div>
    </div>
    <script>
        // Ensure the legend appears open by default
        document.getElementById('legend-header').addEventListener('click', function() {
            const content = document.getElementById('legend-content');
            if (content.style.display === 'none') {
                content.style.display = 'block';
                this.innerHTML = 'Temperature ▼';
            } else {
                content.style.display = 'none';
                this.innerHTML = 'Temperature ▲';
            }
        });
    </script>
    """

    weather_map.get_root().html.add_child(folium.Element(legend_html))

    title_html = f"""
        <h3 align="center" style="font-size:15px"><b>Weather Map Averages - {selected_month}</b></h3>
    """
    weather_map.get_root().html.add_child(folium.Element(title_html))
    
    
   # Add revolving icon
    revolving_icon_html = """
                           <div style="
        position: fixed; top: 70px; right: 20px; 
        width: 50px; height: 50px; z-index: 1000;">
        <div style="
            width: 50px; height: 50px; 
            border: 5px solid rgba(0, 128, 255, 0.2); 
            border-top: 5px solid rgba(0, 128, 255, 1); 
            border-radius: 50%; 
            animation: spin 1s linear infinite;">
        </div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>



    """

    weather_map.get_root().html.add_child(folium.Element(revolving_icon_html))


    display(weather_map)

    def recenter_map(b):
        weather_map = folium.Map(
            location=[average_weather['latitude'].mean(), average_weather['longitude'].mean()],
            zoom_start=2,
            tiles='OpenStreetMap',
            scrollWheelZoom=True
        )
        update_map(None)  

    recenter_button.on_click(recenter_map)
    


month_dropdown.observe(update_map, names='value')

display(month_dropdown, recenter_button)
update_map(None)

/var/folders/nf/xfcnqlxj30x4djjz1zrhc1qr0000gn/T/ipykernel_75295/1325791526.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  average_weather['last_updated'] = pd.to_datetime(average_weather['last_updated'])


Dropdown(description='Date:', options=('November 2024', 'October 2024', 'August 2024', 'July 2024', 'June 2024…

Button(description='Recenter Map', style=ButtonStyle())